In [1]:
from matplotlib import pyplot as plt
from IPython.display import HTML
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation

from learner import GFO
import pandas as pd
import numpy as np
import yfinance

# Look at spy data since the beginning of 2020
data = pd.DataFrame(yfinance.download('SPY', pd.to_datetime('2020-01-01')))
feature_set = pd.DataFrame(data['Volume'])

feature_set['Adj Close'] = data['Adj Close'] / data['Adj Close'][0]

# Say we want to know what a good combination of the feature EMA12 and the feature
# EMA26 is. We will call these features.
feature_set['EMA12'] = pd.Series.ewm(feature_set['Adj Close'], span=12).mean()
feature_set['EMA28'] = pd.Series.ewm(feature_set['Adj Close'], span=28).mean()

fs = feature_set.pop('Adj Close')
feature_set['Adj Close'] = fs
feature_set = feature_set.dropna()


# Use the first quarter of the dataset to train on
A = np.matrix(feature_set.loc[:, 'EMA12':'EMA28'].head(len(feature_set) // 4))
b = np.matrix(feature_set.loc[:, 'Adj Close'].head(len(feature_set) // 4)).T

# Learn about these features and assign them weights
w, r, steps = GFO(A, b).learn()

print('weights = \n', w)

# Look at how the the resulting vector acts upon the entire dataset
A = np.matrix(feature_set.loc[:, 'EMA12':'EMA28'])
b = np.matrix(feature_set.loc[:, 'Adj Close']).T

# Setup plot and animate
plt.style.use('seaborn-deep')
plt.rcParams["animation.html"] = "jshtml"

fig, ax = plt.subplots(figsize=(22, 5))
line = ax.plot(np.rint(A*steps[0]), label='fit', alpha=0.6)[0]
bl = ax.plot(b, label='wanted')

def animate(i):
    line.set_ydata(A*steps[i])

anim = FuncAnimation(fig, animate, interval=200, frames=A.shape[1])

plt.grid()
plt.axvline(len(feature_set) // 4, label='training limit')
plt.legend(loc=1)
plt.close()

anim

[*********************100%***********************]  1 of 1 completed
optized plane   1 /   2, global_epoch= 2999
optized plane   2 /   2, global_epoch= 3970
weights = 
 [[-0.11622402]
 [ 1.09621154]]
